# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [4]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

spark = SparkSession \
    .builder \
    .appName("Data Frames practice") \
    .getOrCreate()

df = spark.read.json("data/sparkify_log_small.json")


In [5]:
df.createOrReplaceTempView('user_log_table')

In [13]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id ""(empty string) NOT visit?

In [10]:
spark.sql('''
           SELECT DISTINCT page 
           FROM user_log_table
           WHERE userId != ''
           ''').show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [ ]:
# TODO: write your code to answer question 1

# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [ ]:
# TODO: write your code to answer question 3

In [20]:
spark.sql('''
          SELECT count(*)
          FROM user_log_table
          WHERE gender == 'F'
          ''').show()

+--------+
|count(1)|
+--------+
|    3820|
+--------+



# Question 4

How many songs were played from the most played artist?

In [ ]:
# TODO: write your code to answer question 4

In [22]:
spark.sql('''
          SELECT artist, count(*)
          FROM user_log_table
          GROUP BY artist
          ORDER BY 2 DESC
          ''').show()

+--------------------+--------+
|              artist|count(1)|
+--------------------+--------+
|                null|    1653|
|            Coldplay|      83|
|       Kings Of Leon|      69|
|Florence + The Ma...|      52|
|            BjÃÂ¶rk|      46|
|       Dwight Yoakam|      45|
|       Justin Bieber|      43|
|      The Black Keys|      40|
|         OneRepublic|      37|
|                Muse|      36|
|        Jack Johnson|      36|
|           Radiohead|      31|
|        Taylor Swift|      29|
|Barry Tuckwell/Ac...|      28|
|          Lily Allen|      28|
|               Train|      28|
|           Daft Punk|      27|
|           Metallica|      27|
|          Nickelback|      27|
|          Kanye West|      26|
+--------------------+--------+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [ ]:
# TODO: write your code to answer question 5

In [28]:
is_home = spark.sql('''
          SELECT userId, page, ts, CASE WHEN page = 
          'Home' THEN 1 ELSE 0 END AS is_home
          FROM user_log_table
          WHERE(page = 'NextSong') OR (page = 'Home')
          ''')

In [29]:
is_home.createOrReplaceTempView('is_home_table')

In [32]:
cumulative_sum = spark.sql('''
          SELECT *, SUM(is_home) OVER (
          PARTITION BY userId ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
          ) as period
          FROM is_home_table
          ''')

In [34]:
cumulative_sum.createOrReplaceTempView('period_table')

In [35]:
spark.sql("SELECT AVG(count_results) FROM \
          (SELECT COUNT(*) AS count_results FROM period_table \
GROUP BY userID, period, page HAVING page = 'NextSong') AS counts").show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+



In [24]:
# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;
is_home = spark.sql("SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home FROM user_log_table \
            WHERE (page = 'NextSong') or (page = 'Home') \
            ")

# keep the results in a new view
is_home.createOrReplaceTempView("is_home_table")

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql("SELECT *, SUM(is_home) OVER \
    (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
    FROM is_home_table")

# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

# find the average count for NextSong
spark.sql("SELECT AVG(count_results) FROM \
          (SELECT COUNT(*) AS count_results FROM period_table \
GROUP BY userID, period, page HAVING page = 'NextSong') AS counts").show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

